In [1]:
import sys, glob
import numpy as np
import matplotlib.pyplot as plt

plt.rc('font', size=40)
plt.rc('lines', lw=4)

files = np.array(glob.glob('./*spec*.dat')) # load in sample spectrum file, relevant to data in Fig. 4 of https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.013046
print("Loaded files:")
print(files)

Loaded files:
['./Run_TP_dyn_all_lanth_wind2_all_md0.097050_vd0.197642_mw0.083748_vw0.297978_spec_2020-11-03.dat']


In [ ]:
def get_params(string):
    name_parse = string.split('/')[-1].split('_')
    params = np.array([name_parse[1], name_parse[5][-1], name_parse[7][2:], name_parse[8][2:], name_parse[9][2:], name_parse[10][2:]])
    return params

cutoff = 60 # takes only the first ~60 time steps in each spectrum which represents the first 22.6 days

for idx in range(files.shape[0]): # for every spectrum in our library of spectra
    params = get_params(files[idx]) # parse filenames into parameters, these will be used as interpolation inputs
    f = open(files[idx], 'r')
    lines = f.readlines()
    if idx == 0:
        for line in lines:
            if line[0] == '#': # finding all lines which are block headers and contain time information
                try:
                    times_spec = np.append(times_spec, float(line.split()[-1])) # obtaining time values for each spectrum
                except NameError:
                    times_spec = np.array(float(line.split()[-1])).reshape(1)
                if len(times_spec) == cutoff: break
    print("Times at which spectra exist:")
    print(times_spec, "\n")
    f.close()

    print("Loading spectrum %d of %d \n" % (idx+1, files.shape[0]))
    spec_data = np.loadtxt(files[idx])
    print("Original shape of spectrum: ", spec_data.shape)
    print("First index is 66 days * 1024 wavelength bins per day = 67584")
    print("Second index is lower wav bin limit + upper wav bin limit + 54 angle indices + 54 angle uncertainty indices = 110 \n")
    spec_data = np.array(np.split(spec_data, spec_data.shape[0]/1024)) # splitting the spectra into a shape that corresponds to [simulation index, wavelength index, angular bin index (0-53) or angular bin uncertainty index (54-107)]
    print("Shape of spectrum after re-shaping into blocks of size 1024 corresponding to 1024 wavelength bins per spectrum:")
    print(spec_data.shape)
    print("66 times (in days, given by times_spec), 1024 wavelength bins (given by wav_lower/wav_upper below in code), same 110 indices as above \n")
    wav_lower = spec_data[0, :, 0] # lower limits for each wavelength bin
    wav_upper = spec_data[0, :, 1] # upper limits for each wavelength bin
    spec = spec_data[:cutoff, :, 2:] # taking only the first [cutoff] time samples and ignoring the iteration numbers and time values (taken care of in code block above)
    print("Final spectrum shape: ", spec.shape)
    print("60 times (due to cutoff), same 1024 wavelength bins, 108 angular indices (54 angles + 54 uncertainties) \n")
    if (spec.shape[0] != 60) or (spec.shape[1] != 1024) or (spec.shape[2] != 108):
        print('skipped ', params, " due to ill-fitting spectrum shape")
        continue

    try:
        params_all = np.append(params_all, params[None, :], axis=0) # storing all parameter values
        spec_all = np.append(spec_all, spec[None, :], axis=0) # storing all spectra values
    except NameError:
        params_all = params[None, :]
        spec_all = spec[None, :]
    
    print("Spectrum index \t Number time steps in spectrum \t Final spectrum time (days) \t Spectrum shape")
    print(idx, "\t \t", times_spec.shape[0], "\t \t \t \t", times_spec[-1], "\t \t \t", spec.shape)

In [23]:
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.4e' % x)

def spec_file_to_dataframe(files, cutoff=60):
    
    wavs = ['Wavelength Lower Limit', 'Wavelength Upper Limit']
    angles = np.degrees([np.arccos((1 - (i-1)*2/54.)) for i in np.arange(1, 55, 1)])
    angle_names = ['Spectral Flux Density at theta = %3.2f' % (i) for i in angles]
    angle_uncertainty_names = ['Spectral Flux Density Uncertainty at theta = %3.2f' % (i) for i in angles]
    names = np.copy(wavs).tolist()
    names.extend(angle_names)
    names.extend(angle_uncertainty_names)
    
    dfs = {}
    
    for file_idx in range(files.shape[0]):
        
        dfs[str(file_idx)] = {}

        f = open(files[file_idx], 'r')
        lines = f.readlines()
        if file_idx == 0:
            for line in lines:
                if line[0] == '#': # finding all lines which are block headers and contain time information
                    try:
                        times_spec = np.append(times_spec, float(line.split()[-1])) # obtaining time values for each spectrum
                    except NameError:
                        times_spec = np.array(float(line.split()[-1])).reshape(1)
                    if len(times_spec) == cutoff: break
        f.close()
        
        spec_data = np.loadtxt(files[file_idx])
        spec_data = np.array(np.split(spec_data, spec_data.shape[0]/1024))
        
        for time_idx in range(times_spec.shape[0]):

            df = pd.DataFrame(spec_data[time_idx])
            df.columns = names

            dfs[str(file_idx)][str(time_idx)] = {}
            dfs[str(file_idx)][str(time_idx)]["time"] = times_spec[time_idx]
            dfs[str(file_idx)][str(time_idx)]["data"] = df
            
            #pd.DataFrame({'time': times_spec[time_idx], 'data': df})
            
    dfs = pd.DataFrame(dfs)

    return dfs

dfs = spec_file_to_dataframe(files)

print(dfs.iloc[2].iloc[0])

{'time': 0.297, 'data':       Wavelength Lower Limit  Wavelength Upper Limit  \
0                 1.0000e-05              1.0047e-05   
1                 1.0047e-05              1.0095e-05   
2                 1.0095e-05              1.0143e-05   
3                 1.0143e-05              1.0191e-05   
4                 1.0191e-05              1.0240e-05   
...                      ...                     ...   
1019              1.2500e-03              1.2560e-03   
1020              1.2560e-03              1.2619e-03   
1021              1.2619e-03              1.2679e-03   
1022              1.2679e-03              1.2739e-03   
1023              1.2739e-03              1.2800e-03   

      Spectral Flux Density at theta = 0.00  \
0                                4.3010e-03   
1                                1.4458e-04   
2                                1.0453e-04   
3                                1.2412e-04   
4                                1.0976e-04   
...                  

In [ ]:
params_all = params_all[:, 2:].astype('float')

time_idx = 50 # trains the NN for *ONE* time point to reduce dimensionality for initial test, free this constraint when we get a good interpolation working

wavs = np.logspace(np.log10(wav_lower[0]), np.log10(wav_lower[-1]), num=1024) # wavelengths are binned into 1024 bins
spectrum = spec_all[:, time_idx, :, 0] # taking all the samples, at the specified time index, considering all wavelengths, taking the first angular bin (0-15.64 degrees)
spectrum[np.where(spectrum <= 0)] = np.min(spectrum[np.nonzero(spectrum)])/100 # remove negative/zero values for well behaved log
spectrum = np.log10(spectrum)

test_idx = 258 # removing one spectrum from the dataset to use as a test case
test_params = params_all[test_idx]
test_spectrum = spectrum[test_idx]
params_all = np.delete(params_all, test_idx, axis=0)
spectrum = np.delete(spectrum, test_idx, axis=0)
spec_err = np.ones_like(spectrum)*0.01 # need some uncertainty to use the chi2 likelihood function, assume low error for now, apply more realistic value later

print(params_all.shape, spectrum.shape)

intp_nn = senni.Interpolator(params_all, spectrum, spec_err, frac=0.1, test_frac=0.1, hlayer_size=8, p_drop=0.05,
                   epochs=1000, learning_rate=1e-1, betas=(0.9, 0.99), eps=1e-2, weight_decay=1e-6,
                   epochs_per_lr=200, lr_divisions=5, lr_frac=1./3., batch_size=128, shuffle=True, 
                   working_dir='.', loss_func='mape', no_pad=True) # set up NN interpolator, most of these will not be adjusted

intp_nn.train() # train NN

intp_nn.load('models/bestmodel.pt') # load best model achieved during training

for i in range(1000): # we want many evaluations to see how the effects of dropout will quantify uncertainty
    prediction = intp_nn.evaluate(test_params[None, :], eval_mode=False)
    try:
        predictions = np.append(predictions, prediction, axis=0)
    except NameError:
        predictions = prediction
    del prediction

print(predictions.shape)

intp_spectrum_nn = np.mean(predictions, axis=0) # take mean of 1000 evaluations to get mean spectrum
intp_spectrum_nn_err = np.std(predictions, axis=0) # std of 1000 evaluations to get uncertainty from dropout
#intp_spectrum_nn = intp_nn.evaluate(test_params[None, :])

intp_rf = slick.Interpolator() # train with random forest which typically has better convergence to verify fidelity of NN output

intp_rf.train(params_all, spectrum) # actual training with RF

intp_spectrum_rf = intp_rf.evaluate(test_params[None, :]) # prediction from RF

print(times_spec[time_idx])

np.savetxt('spectrum_intp_nn.dat', np.c_[wavs, test_spectrum, intp_spectrum_nn.flatten(), intp_spectrum_nn_err.flatten()]) # save NN outputs along with uncertainty
np.savetxt('spectrum_intp_rf.dat', np.c_[wavs, test_spectrum, intp_spectrum_rf.flatten()]) # RF outputs with no uncertainty, only used for NN fidelity
